In [1]:
import subprocess
import shlex
import re

import numpy as np

from IPython.display import display, Markdown

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# PractRand (Practically Random) - statistical tests for RNGs

In [3]:
def test_randomness_practrand(
    mode, seeds=[42], limit='1GB',
):
    outputs = []
    
    for seed in seeds:
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release -- --seed {seed} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        practrand = subprocess.run(
            shlex.split(f"./practrand stdin64 -tlmax {limit}"), stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        outputs.append(practrand.stdout)
        
    most_suspect = None
    
    for output in outputs:
        failures = output.count("FAIL")
        suspicions = output.count("SUSPICIOUS") * 2 + output.count("suspicious")
        unusuals = output.count("unusual")
        
        suspicion = failures * 10 + suspicions * 2 + unusuals
        
        if (most_suspect is None) or (suspicion > most_suspect[0]):
            most_suspect = (suspicion, output)

    if "FAIL" in most_suspect[1]:
        display(Markdown(f"## <span style='color:purple'><u>{mode}</u></span>"))
    elif "SUSPICIOUS" in most_suspect[1]:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    elif "suspicious" in most_suspect[1]:
        display(Markdown(f"## <span style='color:orange'>*{mode}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}\n* limit: {limit}"))

In [4]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=1)
    
    test_randomness_practrand(mode, seeds=seeds, limit='1GB')

## <span style='color:green'>Monolithic</span>

#### Test output (most suspect):

>RNG_test using PractRand version 0.95
>RNG = RNG_stdin64, seed = unknown
>test set = core, folding = standard (64 bit)
>
>rng=RNG_stdin64, seed=unknown
>length= 128 megabytes (2^27 bytes), time= 3.8 seconds
>  no anomalies in 196 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 256 megabytes (2^28 bytes), time= 10.7 seconds
>  no anomalies in 213 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 512 megabytes (2^29 bytes), time= 20.2 seconds
>  no anomalies in 229 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 1 gigabyte (2^30 bytes), time= 40.4 seconds
>  no anomalies in 246 test result(s)
>
>

#### Parameters:

* seeds: [5807882503286178200]
* limit: 1GB

...

## <span style='color:green'>IndependentSimulationNoDispersal</span>

#### Test output (most suspect):

>RNG_test using PractRand version 0.95
>RNG = RNG_stdin64, seed = unknown
>test set = core, folding = standard (64 bit)
>
>rng=RNG_stdin64, seed=unknown
>length= 128 megabytes (2^27 bytes), time= 3.5 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low16/64]BCFN(2+9,13-8,T)        R= +16.8  p =  3.3e-5   unusual          
>  ...and 195 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 256 megabytes (2^28 bytes), time= 8.9 seconds
>  no anomalies in 213 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 512 megabytes (2^29 bytes), time= 18.8 seconds
>  no anomalies in 229 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 1 gigabyte (2^30 bytes), time= 34.8 seconds
>  no anomalies in 246 test result(s)
>
>

#### Parameters:

* seeds: [10858861011221792565]
* limit: 1GB